# IoT Traffic Anomaly Detection Reproduction

This notebook reproduces the workflow from Vigoya et al. (Electronics 2021) using proxy IoT/SCADA datasets. It mirrors the paper's preprocessing (cyclical time encoding, binning, one-hot encoding, flow aggregation), nested CV with SMOTE, RFE feature selection, and evaluation of five shallow ML models. Optional sections include extended models and threat-intel similarity.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

from iot_anomaly_detection.data import load_hf_dataset
from iot_anomaly_detection.data.feature_mapping import infer_feature_mapping, apply_feature_mapping
from iot_anomaly_detection.data.preprocessing import FeaturePreprocessor
from iot_anomaly_detection.models import (
    LogisticRegressionDetector,
    BernoulliNBDetector,
    RandomForestDetector,
    AdaBoostDetector,
    LinearSVMDetector,
    ExtraTreesDetector,
    GradientBoostingDetector,
    nested_cv_evaluate,
)
from iot_anomaly_detection.utils.cv import build_nested_cv
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE
from sklearn.impute import SimpleImputer
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

sns.set_theme(style="whitegrid")
OUTPUT_DIR = Path("notebooks/outputs")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)


In [ ]:
# Config: select dataset and sample size.
dataset_name = "vossmoos/vestasv52-scada-windturbine-granada"
sample_size = 5000
random_state = 42
rfe_features = 10
use_extra_models = False
run_threat_intel = False

# Other available datasets:
# - "fenar/iot-security"
# - "schooly/Cyber-Security-Breaches"
# - "stu8king/securityincidents"
# - "kutay1907/scadaphotodataset"
# - "kutay1907/ScadaData100k"


In [ ]:
# Load a proxy dataset from Hugging Face.
df = load_hf_dataset(dataset_name, split="train", sample_size=sample_size)
df.head()


In [ ]:
# Infer and inspect feature mapping against the core attributes.
mapping = infer_feature_mapping(df.columns)
mapping


In [ ]:
# Build labels and show class balance after mapping.
mapped = apply_feature_mapping(df, mapping)
label_counts = mapped["label"].value_counts()
label_counts


In [ ]:
# Plot label distribution.
plt.figure(figsize=(4, 3))
sns.barplot(x=label_counts.index.astype(str), y=label_counts.values, palette="viridis")
plt.title("Label Distribution")
plt.xlabel("Label")
plt.ylabel("Count")
plt.tight_layout()
plt.savefig(OUTPUT_DIR / "label_distribution.png", dpi=200)
plt.show()


In [ ]:
# Evaluate accuracy vs number of features selected by RFE for a baseline model.
feature_counts = [5, 8, 10, 12, 14]
X = df
y = mapped["label"]

acc_means = []
for k in feature_counts:
    preprocessor = FeaturePreprocessor(mapping=mapping)
    summary, _, _, _ = nested_cv_evaluate(
        X,
        y,
        LogisticRegressionDetector(),
        preprocessor,
        rfe_features=k,
    )
    acc_means.append(summary["accuracy"]["mean"])

plt.figure(figsize=(6, 4))
plt.plot(feature_counts, acc_means, marker="o")
plt.title("Accuracy vs Selected Features (RFE)")
plt.xlabel("Number of Features")
plt.ylabel("Accuracy")
plt.tight_layout()
plt.savefig(OUTPUT_DIR / "accuracy_vs_features.png", dpi=200)
plt.show()


In [ ]:
# Train and evaluate the five shallow models with nested CV.
models = [
    LogisticRegressionDetector(),
    BernoulliNBDetector(),
    RandomForestDetector(),
    AdaBoostDetector(),
    LinearSVMDetector(),
]

if use_extra_models:
    models += [ExtraTreesDetector(), GradientBoostingDetector()]

leaderboard_rows = []
model_results = {}

for model in models:
    preprocessor = FeaturePreprocessor(mapping=mapping)
    summary, _, _, fold_predictions = nested_cv_evaluate(
        X,
        y,
        model,
        preprocessor,
        rfe_features=rfe_features,
    )
    model_results[model.name] = {"summary": summary, "fold_predictions": fold_predictions}
    leaderboard_rows.append({
        "model": model.name,
        "accuracy_mean": summary["accuracy"]["mean"],
        "accuracy_std": summary["accuracy"]["std"],
        "precision_mean": summary["precision"]["mean"],
        "precision_std": summary["precision"]["std"],
        "recall_mean": summary["recall"]["mean"],
        "recall_std": summary["recall"]["std"],
        "f1_mean": summary["f1"]["mean"],
        "f1_std": summary["f1"]["std"],
    })

leaderboard = pd.DataFrame(leaderboard_rows).sort_values(by="f1_mean", ascending=False)
leaderboard


In [ ]:
# Save leaderboard table as an image.
fig, ax = plt.subplots(figsize=(8, 0.5 + 0.4 * len(leaderboard)))
ax.axis("off")
table = ax.table(
    cellText=np.round(leaderboard.drop(columns=["model"]).values, 3),
    colLabels=leaderboard.drop(columns=["model"]).columns,
    rowLabels=leaderboard["model"],
    loc="center",
)
table.auto_set_font_size(False)
table.set_fontsize(8)
plt.tight_layout()
plt.savefig(OUTPUT_DIR / "leaderboard_table.png", dpi=200)
plt.show()


In [ ]:
# Plot model comparison (Accuracy, Precision, Recall, F1).
metrics = ["accuracy_mean", "precision_mean", "recall_mean", "f1_mean"]
melted = leaderboard.melt(id_vars=["model"], value_vars=metrics, var_name="metric", value_name="score")
plt.figure(figsize=(8, 4))
sns.barplot(data=melted, x="model", y="score", hue="metric")
plt.title("Model Comparison (Mean Metrics)")
plt.xticks(rotation=30, ha="right")
plt.tight_layout()
plt.savefig(OUTPUT_DIR / "model_comparison.png", dpi=200)
plt.show()


In [ ]:
# Nested CV with ROC curves and validation AUC (inner CV).
def nested_cv_scores(X, y, model, mapping, rfe_features=10, random_state=42):
    outer_cv, inner_cv = build_nested_cv(random_state=random_state)
    y_true_all = []
    y_score_all = []
    inner_scores = []

    param_grid = {f"model__{k}": v for k, v in model.get_search_space().items()}

    for train_idx, test_idx in outer_cv.split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        preprocessor = FeaturePreprocessor(mapping=mapping)
        pipeline = Pipeline(
            steps=[
                ("preprocess", preprocessor),
                ("impute", SimpleImputer(strategy="median")),
                ("smote", SMOTE(random_state=random_state)),
                ("rfe", RFE(estimator=DecisionTreeClassifier(random_state=random_state), n_features_to_select=rfe_features, step=0.1)),
                ("model", model.build_estimator(model.get_default_params())),
            ]
        )
        search = GridSearchCV(pipeline, param_grid=param_grid, cv=inner_cv, scoring="roc_auc", n_jobs=-1)
        search.fit(X_train, y_train)
        inner_scores.append(search.best_score_)
        best_model = search.best_estimator_

        if hasattr(best_model, "predict_proba"):
            y_score = best_model.predict_proba(X_test)[:, 1]
        elif hasattr(best_model, "decision_function"):
            y_score = best_model.decision_function(X_test)
        else:
            y_score = best_model.predict(X_test)
        y_true_all.append(y_test.to_numpy())
        y_score_all.append(np.asarray(y_score))

    return np.concatenate(y_true_all), np.concatenate(y_score_all), inner_scores

roc_records = []
validation_records = []
plt.figure(figsize=(7, 5))
for model in models:
    y_true_all, y_score_all, inner_scores = nested_cv_scores(X, y, model, mapping, rfe_features=rfe_features)
    fpr, tpr, _ = roc_curve(y_true_all, y_score_all)
    roc_auc = auc(fpr, tpr)
    roc_records.append({"model": model.name, "roc_auc": roc_auc})
    validation_records.append({"model": model.name, "val_auc_mean": np.mean(inner_scores), "val_auc_std": np.std(inner_scores, ddof=1)})
    plt.plot(fpr, tpr, label=f"{model.name} (AUC={roc_auc:.3f})")

plt.plot([0, 1], [0, 1], linestyle="--", color="gray")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curves (Nested CV)")
plt.legend(fontsize=8)
plt.tight_layout()
plt.savefig(OUTPUT_DIR / "roc_curves.png", dpi=200)
plt.show()

validation_df = pd.DataFrame(validation_records)
plt.figure(figsize=(7, 4))
plt.bar(validation_df["model"], validation_df["val_auc_mean"], yerr=validation_df["val_auc_std"], capsize=4)
plt.xticks(rotation=30, ha="right")
plt.ylabel("Validation ROC AUC (mean ± std)")
plt.title("Inner CV Validation Performance")
plt.tight_layout()
plt.savefig(OUTPUT_DIR / "validation_auc.png", dpi=200)
plt.show()


## Notes
- If your dataset has missing fields, the mapper simulates required attributes (protocol, ports, timestamps) so the pipeline can run end-to-end.
- Expect tree-based models (Random Forest, AdaBoost) to outperform linear baselines, aligning with the paper's findings.
- Charts are saved to `notebooks/outputs/` for easy inclusion in reports.
